在本章中，我们将学到如下内容：
+ 介绍常用分类算法的概念
+ 使用scikit-learn机器学习库
+ 选择机器学习算法时需要注意的问题  

5  
# 3.1 分类算法的选择
再次借用一下“没有免费午餐”理论：没有任何一种分类器可以在所有可能的应用场景下都有良好的表现  
总而言之，分类器的性能、计算能力和预测能力，在很大程度上都依赖于用于模型训练的相关数据。训练机器学习算法所涉及的五个主要步骤可以概述如下：  
1. 特征的选择  
2. 确定性能评价标准
3. 选择分类器及其优化算法
4. 对模型性能的评估
5. 算法的调优
5  

# 3.2 初涉scikit-learn的使用
## 使用sckit-learn训练感知器

In [ ]:
'''
python	sklearn	datasets	load_iris() feature_names
python	sklearn	datasets	load_iris() target
python	sklearn	datasets	load_iris() data
python	sklearn	datasets	load_iris() target_names
'''
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
print(iris.feature_names)
print(iris.target_names)

为了评估训练得到的模型在未知数据上的表现，我们进一步将数据集划分为训练数据集和测试数据集 

In [ ]:
'''
python	sklearn	model_selection	train_test_split()
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 0)

在此，我们将使用scikit-learn的preprocessing模块中的standardscaler类对特征进行标准化处理  

In [ ]:
'''
5
'''
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() 
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

需要注意的是，我们要使用相同的缩放参数分别处理训练和测试数据集，以保证它们的值是彼此相当的  
在对训练数据做了标准化处理后，我们现在可以训练感知器模型了。scikit-learn中的大多数算法本身就使用一对多方法来支持多类别分类，因此，我们可以将三种鸢尾花的数据同时输入到感知器中    

In [ ]:
'''
python	sklearn	Linear Models	Perceptron() max_iter
python	sklearn	Linear Models	Perceptron() eta0
'''
from sklearn.linear_model import Perceptron
ppn = Perceptron(max_iter = 40, eta0 = 0.1, random_state = 0)
ppn.fit(X_train_std, y_train)

此模型的参数eta0与我们自行实现的感知器中的学习速率eta等价，而参数max_iter定义了迭代的次数  
与第2章中实现的感知器一样，使用scikit-learn完成模型的训练后，就可以在测试数据集上使用predict方法进行预测了 5 

In [ ]:
'''
python	str	format()	format() d
'''
y_pred = ppn.predict(X_test_std)
print('Misclassified samples: {:d}'.format((y_test != y_pred).sum()))

许多机器学习从业者通常使用准确率而不是误分类来评判一个模型，它们关系如下：  
1 - 误分类率 = 准确率  
可以通过metrics模块计算感知器在测试数据集上的分类准确率：  

In [ ]:
'''
python	sklearn	Classification metrics	accuracy_score()
'''
from sklearn.metrics import accuracy_score
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))

绘制刚刚训练过的模型的决策区域，我们做少许修改：使用小圆圈来高亮显示来自测试数据集的样本 5  

In [ ]:
'''
python	numpy	Array manipulation routines	unique()
python	matplotlib	colors	ListedColormap()
python	numpy	The N-dimensional array (ndarray)	min()
python	numpy	The N-dimensional array (ndarray)	max()
python	numpy	Array creation routines	meshgrid()
python	matplotlib	Pyplot function overview	contourf()
python	matplotlib	Pyplot function overview	scatter() alpha
python	matplotlib	collections	label
'''

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier,
                          test_idx = None, resolution = 0.02):
    # 设置散点生成器和色图
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # 绘制决策边界
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                          np.arange(x2_min, x2_max, resolution)) 
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha = 0.4, cmap = cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    # 绘制样本分类
    X_test, y_test = X[test_idx, :], y[test_idx]
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x = X[y == cl, 0], y = X[y == cl, 1],
                   alpha = 0.8, color = cmap(idx),
                   marker = markers[idx], label = cl)

    # 高亮测试样本
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        plt.scatter(X_test[:, 0], X_test[:, 1], c = 'yellow',
                   alpha = 0.5, linewidth = 1, marker = 'o',
                   s = 55, label = 'test set')

In [ ]:
'''
python	numpy	Array manipulation routines	vstack()
python	numpy	Array manipulation routines	hstack()
python	matplotlib	Pyplot function overview	legend() loc

'''
import numpy as np
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))
plot_decision_regions(X = X_combined_std,
                     y = y_combined,
                     classifier = ppn,
                     test_idx = range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc = 'upper left')
plt.show()

从结果呈现的图像中我们可以看到，无法通过一个线性决策边界完美区分三类样本  
对于无法完美线性可分的数据集，感知器算法将永远无法收敛，这也是在实践中一般不推荐使用感知器算法的原因 5
# 3.3 逻辑斯蒂回归中的类别概率
感知器在样本不是完全线性可分的情况下不会收敛的原因：在每次迭代过程中，总是存在至少一个分类错误的样本，从而导致额权重持续更新  
为了提高分类的效率，我们学习另外一种针对线性二类别分类问题的简单但更高效的算法：逻辑斯蒂回归 5  
## 3.3.1 初识逻辑斯蒂回归与条件概率
与感知器及Adaline类似，逻辑斯蒂回归模型也是适用于二类别分类问题的线性模型，通过一对多技术可以扩展到多类别分类  
我们先介绍一下几率比，它指的是特定事件发生的几率。用数学公式表示为$\frac{p}{(1-p)}$，其中p为正事件发生的概率  
正事件是指我们所要预测的事件，以一个患者患有某种疾病的概率为例，我们可以将正事件的类标标记为y=1。更进一步，我们可以定义logit函数，它是几率比的对数函数(log-odds, 对数几率)  
$$logit(p)=log\frac{p}{(1-p)}$$
logit函数的输入值范围介于区间[0,1]，它能将输入转换到整个实数范围内，由此可以将度数几率记为输入特征值的线性表达式 5  
$$logit(p(y=1|x))=w_0x_0+w_1x_1+\dots+w_mx_m=\sum_{i=0}^n{w_mx_m}=w^Tx$$
此处，$p(y=1|x)$是在给定特征x的条件下，某一个样本属于类别1的条件概率  
我们在此的真正目的是预测某一样本属于特定类别的概率，它是logit函数的反函数，也称作logistic函数，由于它的图像呈S形，因此有时也简称sigmoid函数  
$$\phi(z)=\frac{1}{1+e^{-z}}$$
这里的z为净输入，也就是样本特征与权重的线性组合，其计算方式为： 5  
$$z=w^Tx=w_0x_0+w_1x_1+\dots+w_mx_m$$
我们来绘制一下自变量取值介于区间[-7,7]的sigmoid函数的图像  

In [ ]:
'''
python	numpy	Mathematical functions	exp()
python	matplotlib	Pyplot function overview	axvline()
python	matplotlib	lines	color
python	matplotlib	Pyplot function overview	axhspan()
python	matplotlib	patches	Patch() facecolor
python	matplotlib	patches	Patch() alpha
python	matplotlib	patches	Patch() linestyle
python	matplotlib	Pyplot function overview	axhline()
python	matplotlib	lines	linestyle

'''
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
z = np.arange(-7, 7, 0.1)
phi_z = sigmoid(z)
plt.plot(z, phi_z)
plt.axvline(0.0, color='k')
plt.axhspan(0.0, 1.0, facecolor='1.0', alpha=1.0, linestyle='dotted')
plt.axhline(y=0.5, linestyle='dotted', color='black')
plt.axhline(y=0, linestyle='dotted', color='black')
plt.axhline(y=1.0, linestyle='dotted', color='black')
plt.yticks([0.0, 0.5, 1.0])
plt.ylim(-0.1, 1.1)
plt.xlabel('z')
plt.ylabel('$\phi (z)$')
plt.show()

执行上述代码，将会呈现一个S形曲线  
可以看到，当z趋向于无穷大时($z\to\infty$)时，$\varphi(z)$趋近于1，这是由于$e^{-z}$在z值极大的情况下其值变得极小。类似的，反之亦然 5  
由此，可以得出结论：sigmoid函数以实数值作为输入并将其映射到[0,1]区间，其拐点位于$\varphi(z)=0.5$处  
在Adaline中，我们使用恒等函数$\varphi(z)=z$作为激励函数。而在逻辑斯蒂回归中，只是简单地将前面提及的sigmoid函数作为激励函数，如下图所示：  
![3-1](../syn_pic/py_machine_learning/3-1.png)
在给定特征x及其权重w的情况下，sigmoid函数的输出给出了特定样本x属于类别1的概率$\varphi(z)=P(y=1|x;w)$  
预测得到的概率可以通过一个量化器(单位阶跃函数)简单地转换为二元输出： 5  
$$\hat{y}=\begin{cases}1&若\phi(z)\ge0.5 \\ 0&其他\end{cases}$$
对照前面的sigmoid函数的图像，它其实相当于：
$$\hat{y}=\begin{cases}1&若z\ge0.0 \\ 0&其他\end{cases}$$
对于许多应用实践来说，我们不但对类标预测感兴趣，而且对事件属于某一类别的概率进行预测也非常有用 5  
## 3.3.2 通过逻辑斯蒂回归模型的代价函数获得权重  
在上一章中，我们将其代价函数定义为误差平方和(SSE)  
$$J(w)=\sum_i{\frac{1}{2}(\phi(z^{(i)}-y^{(i)}))^2}$$
通过最小化此代价函数，我们可以得到Adaline分类模型的权重w。为了推导出逻辑斯蒂回归模型的代价函数，我们需要先定义一个**最大似然函数**L，假定数据集中的每个样本都是相互独立的，其计算公式如下：    
$$L(w)=P(y|x;w)=\prod_{i=1}^n{P(y^{(i)}|x^{(i)};w)}=(\phi(z^{(i)}))^{y^{(i)}}(1-\phi(z^{(i)}))^{1-y^{(i)}} $$
### 问题：什么是最大似然函数？
> 相关最大似然估计（MLE），属于参数估计技术，统计推断的一种  
可参考“数理统计学教程”陈希孺 5

在实际应用中，很容易对此方程的自然对数进行最大化处理，故定义了对数似然函数： 5  
$$l(w)=logL(w)=\sum_{i=1}^n{log(\phi(z^{(i)}))+(1-y^{(i)})log(1-\phi(z^{(i)}))}$$
可以通过梯度上升等优化算法最大化似然函数。改写一下对数似然函数作为代价函数J  
$$J(w)=\sum_{i=1}^n{-log(\phi(z^{(i)}))-(1-y^{(i)})log(1-\phi(z^{(i)}))}$$
为了更好理解这一代价函数，先了解一下如何计算单个样本示例的成本：  
$$J(\phi(z),y;w)=-ylog(\phi(z))-(1-y)log(1-\phi(z))$$
5  
如果y=0，则第一项为0;如果y=1，则第二项为0：  
$$J(\phi(z),y;w)=\begin{cases} -log(\phi(z))&若y=1 \\ -log(1-\phi(z))&若y=0 \end{cases}$$
下图解释了在不同$\varphi(z)$值时对单一示例样本进行分类的代价： 
![3-2](../syn_pic/py_machine_learning/3-2.png)
如果将样本正确划分到类别1中，代价将趋近于0（实线）。类似地，如果正确将样本划分到类别0中，y轴所代表的代价也将趋近于0（虚线） 5  
然而，如果分类错误，代价将趋近于无穷，这也就意味着错误预测带来的代价将越来越大 5  
## 3.3.3 使用scikit-learn训练逻辑斯蒂回归模型
$$J(w) = \frac{1}{2}\sum_{i}{(y^{(i)}-\phi(z^{(i)}))^2}$$
如果想要自己编写代码实现逻辑斯蒂回归，只要将第2章中的代价函数J（如上所示）替换为下面的代价函数：  
$$J(w)=-\sum_{i=1}^n{log(\phi(z^{(i)}))+(1-y^{(i)})log(1-\phi(z^{(i)}))}$$
由于scikit-learn已经有现成的经过高度优化的逻辑斯蒂算法，而且还支持多类别分类，因此，我们使用sklearn.linear_model.LogisticRegression类来完成  

In [ ]:
'''
5
python	sklearn	Linear Models	LogisticRegression() C

'''
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000.0, random_state=0) # C 正则化参数
lr.fit(X_train_std, y_train)
plot_decision_regions(X = X_combined_std,
                     y = y_combined,
                     classifier = lr,
                     test_idx = range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc = 'upper left')
plt.show()

可以通过predict_proba方法来预测样本属于某一类别的概率  

In [ ]:
'''
python	sklearn	Linear Models	LogisticRegression() predict_proba()
python	numpy	The N-dimensional array (ndarray)	round()
'''
lr.predict_proba(X_test_std[-2:,:]).round(3)

此数组表明模型预测此样本属于Iris-setosa的概率为99.4%，属于Iris-Virginica的概率为0.6%，属于Iris-Versicolor的概率为0%  
**逻辑斯蒂回归中的回归系数更新用到的梯度下降本质上与第2章中的公式是相同的**。证明如下，首先，计算对数似然函数对第j个权重的偏导：  
$$\frac{\partial}{\partial{w_j}}l(w) = \left(y\frac{1}{\phi(z)}-(1-y)\frac{1}{1-\phi(z)} \right)\frac{\partial}{\partial{w_j}}\phi(z)$$  
5  
在进入下一步之前，先计算一下sigmoid函数的偏导：  
![3-3](../syn_pic/py_machine_learning/3-3.png)  
我们的目标时求得能够使对数似然函数最大化的权重值，在此需按如下公式更新所有权重：  
$$w_j:=w_j+\eta\sum_{i=1}^n{(y^{(i)}-\phi(z^{(i)}))x^{(i)}}$$
由于我们是同时更新所有的权重的，因此可以将更新规则记为： 5  
$$w:=w+\Delta{w}$$
其中，$\Delta{w}$定义为：  
$$\Delta{w}=\eta\nabla{l(w)}$$
由于最大化对数似然函数等价于最小化前面定义的代价函数J，因此可以将梯度下降的更新规则定义为：  
$$\Delta{w_j}=-\eta\frac{\partial{J}}{\partial{w_j}}=\eta\sum_{i=1}^n{(y^{(i)}-\phi{(z^{(i)})x^{(i)}})}$$
$$w:=w+\Delta{w},\Delta{w}=-\eta\nabla{J(w)}$$
5  
这等价于第2章中的梯度下降规则 5  
## 3.3.4 通过正则化解决过拟合问题  
过拟合是机器学习中的常见问题，它是指模型在训练数据集上表现良好，但是用于未知数据时性能不佳  
如果一个模型出现了过拟合问题，我们也说此模型有高方差，这有可能是因为使用了相关数据中过多的参数，从而使得模型变得国于复杂。同样，模型也可能面临欠拟合（高偏差）问题  
![3-4](../syn_pic/py_machine_learning/3-4.png)  
如果我们多次重复训练一个模型，如使用训练数据集中不同的子集，方差可以用来衡量模型对特定样本实例预测的一致性。可以说模型对训练数据中的随机性是敏感的  
相反，当我们在不同的训练数据集上多次重建模型时，偏差可以从总体上衡量预测值与实际值之间的差异;偏差并不是由样本的随机性导致的，它衡量的是系统误差 5  
偏差-方差权衡就是通过正则化调整模型的复杂度。正则化是解决共线性（特征间高度相关）的一个很有用的方法，它可以过滤掉数据中的噪音，并最终防止过拟合  
### 问题：为什么正则化可以防止过拟合？
> 由于过拟合本质是过多的特征被启用导致的，导致模型泛化性变差，所以防止过拟合要降低特征的数量，可以通过使w个数减少，问题就变成让W向量中项的个数最小化，方法就是让w变成或趋近于0，因为向量中0元素对应的x是没有任何权重的 5  
### 问题：什么是L2正则化？
> L2范数是指向量各元素的平方和然后求平方根。我们让L2范数的正则项$\|W\|^2$最小，可以使得W的每个元素都很小，都接近于0，但与L1范数不同，它不会让它等于0，而是接近于0，这里是有很大的区别的哦 5  

正则化背后的概念是引入额外的信息（偏差）来对极端参数权重做出惩罚。最常用的正则化形式被称为L2正则化，有时也称作L2收缩或权重衰减，写作  
$$\frac{\lambda}{2}\|{w}\|^2=\frac{\lambda}{2}\sum_{j=1}^mw_j^2$$
其中，$\lambda$为正则化系数  
**特征缩放之所以重要，其中一个原因就是正则化。为了使得正则化起作用，需要确保所有特征保持统一** 5  
使用正则化方法时，我们只需在逻辑斯蒂回归的代价函数中加入正则化项，以降低回归系数带来的副作用：  
$$J(w)=\left\{\sum_{i=1}^n{-log(\phi(z^{(i)}))+(1-y^{(i)})(-log(1-\phi(z^{(i)})))}\right\}+\frac{\lambda}{2}\|w\|^2$$  
通过正则化系数，保持权值较小时，我们就可以控制模型与训练数据的拟合程度  
前面用到了sklearn中的LogisticRegression类，其中的参数C来自支持向量机中的约定，它时正则化系数的倒数：  
$$C=\frac{1}{\lambda}$$
5  
由此，我们可以将逻辑斯蒂回归中经过正则化的代价函数写作  
$$J(w)=C\left\{\sum_{i=1}^n{-log(\phi(z^{(i)}))+(1-y^{(i)})(-log(1-\phi(z^{(i)})))}\right\}+\frac{\lambda}{2}\|w\|^2$$
因此，减小正则化参数倒数C的值相当于增加正则化的强度，这可以通过绘制对两个权重系数进行L2正则化后的图像予以展示  

In [ ]:
'''
5
python	Concrete exceptions	ValueError	Integers to negative integer powers are not allowed
python	sklearn	Linear Models	LogisticRegression() C
python	sklearn	Linear Models	LogisticRegression() coef_
python	matplotlib	lines	label
python	matplotlib	Pyplot function overview	xscale()
'''
weights, params = [], []
for c in np.arange(-5, 5):
    lr = LogisticRegression(C=10.0**c, random_state=0) # 原书写着C=10**c 会引起ValueError
    lr.fit(X_train_std, y_train)
    weights.append(lr.coef_[1])
    params.append(10.0**c) # 原书写着C=10**c 会引起ValueError    
weights = np.array(weights)
plt.plot(params, weights[:, 0],
        label='petal length')
plt.plot(params, weights[:, 1], linestyle='--',
        label='petal width')
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.legend(loc='upper left')
plt.xscale('log')
plt.show()

通过结果图像可以看到，如果我们减小参数C的值，也就是增加正则化项的强度，可以导致权重系数逐渐收缩 5  
# 3.4 使用支持向量机最大化分类间隔
另一种性能强大且广泛应用的学习算法是支持向量机，它可以看作对感知器的扩展。在SVM中，我们的优化目标是最大化分类间隔  
此处间隔是指两个分离的超平面间（决策边界）的距离，而最靠近超平面的训练样本称作支持向量，如下图所示：  
![3-5](../syn_pic/py_machine_learning/3-5.png)  
5  
## 3.4.1 对分类间隔最大化的直观认识
决策边界间具有较大的间隔意味着模型具有较小的泛化误差，而较小的间隔则意味着模型可能过拟合  
为了对间隔最大化有个直观认识，我们仔细观察一下两条平行的决策边界，我们分别称其为正负超平面，可表示为：  
$$w_0+w^Tx_{pos}=1$$(1)
$$w_0+w^Tx_{neg}=-1$$(2)  
### 问题：什么是决策边界？
> 待解决的问题


如果我们将等式（1）（2）相减，可以得到： 5  
$$\Rightarrow{w^T(x_{pos}-x_{neg})=2}$$ 
我们可以通过向量w的长度来对其进行规范化，做如下定义：  
$$\|w\|=\sqrt{\sum_{j=1}^m{w_j^2}}$$
由此可以得到如下等式：  
$$\frac{w^T(x_{pos}-x_{neg})}{\|w\|}=\frac{2}{\|w\|}$$
5  
上述等式的左侧可以解释为正、负超平面间的距离，也就是我们要最大化的间隔  
在样本正确划分的前提下，最大化分类间隔也就是使$\frac{2}{\|w\|}$最大化，这也是SVM目标函数，记为：  
$$w_0+w^Tx^{(i)}\ge1若y^{(i)}=1$$
$$w_0+w^Tx^{(i)}\lt-1若y^{(i)}=-1$$
这两个方程可以解释为：所有的负样本都落在负超平面一侧，而所有的正样本则在超平面划分出的区域中。它们可以写成更紧凑的形式： 5  
$$y^{(i)}(w_0+w^Tx^{(i)})\ge1\forall_i$$
通过二次规划的方法，可以对目标函数的倒数项$\frac{1}{2}\|w\|$进行最小化处理 5  
### 问题：什么是二次规划？
> 作者介绍了相关书籍：Vladimir Vapnik的The Nature of Statistical Learning Theory 5  

## 3.4.2 使用松弛变量解决非线性可分问题
> 本章介绍了支持向量机中的松弛变量及其作用：
+ 解决非线性可分数据导致的不收敛
+ 配合C参数取得偏差和方差之间的平衡

对于非线性可分的数据来说，需要放松线性约束条件，以保证在适当的惩罚项成本下，对错误分类的情况进行优化时能够收敛  
取值为正的松弛变量可以简单地加入到线性约束中：5    
$$w^Tx^{(i)}\ge1若y^{(i)}=1-\xi^{(i)}$$
$$w^Tx^{(i)}\lt-1若y^{(i)}=1+\xi^{(i)}$$

由此，在前面提及的约束条件下，新的优化目标为最小化下式：  
$$\frac{1}{2}\|w\|^2+C\left(\sum_i{\xi^{(i)}}\right)$$  
可以使用参数C来控制间隔的大小，进而可以在偏差和方差之间取得平衡，如下图所示： 5  
![3-6](../syn_pic/py_machine_learning/3-6.png)  
我们已经学习了线性SVM的基本概念，现在让我们来训练一个SVM模型以对鸢尾花数据集中的样本进行分类  

In [ ]:
'''

'''
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(X_train_std, y_train)
plot_decision_regions(X_combined_std,
                     y_combined, classifier=svm,
                     test_idx=range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc = 'upper left')
plt.show()

### 逻辑斯蒂回归与支持向量机
> 介绍了线性逻辑斯蒂回归与线性支持向量机的区别和优劣

逻辑斯蒂回归会尽量最大化训练数据集的条件似然，这使得它比支持向量机更易于处理离群点。而支持向量机更关注接近决策边界的点 5  
逻辑斯蒂回归的另一个优势在于模型简单从而更容易实现。此外，逻辑斯蒂回归模型更新方便，适用于处理流数据分析 5  
## 3.4.3 使用scikit-learn实现SVM
> 介绍了专门用于SVM的C++库, 以及在大型数据集中应用随机梯度下降执行SVM、感知器、逻辑斯蒂回归  

与使用原生python代码实现线性分类器相比LIBLINEAR和LIBSVM有齐优点：在用于大型数据时，可以获得极高的训练速度  
然后，有些时候数据集非常巨大以致无法加载到内存中，针对这种情况，scikit-learn提供了SGDClassifier类供用户选择，还通过partial_fit方法支持在线学习  
我们可以使用默认参数以如下方式分别初始化基于随机梯度下降的感知器、逻辑斯蒂回归以及支持向量机模型  

In [ ]:
'''
5
python	sklearn	Linear Models	SGDClassifier() loss
'''
from sklearn.linear_model import SGDClassifier
ppn = SGDClassifier(loss='perceptron')
lr = SGDClassifier(loss='log')
svm = SGDClassifier(loss='hinge')

# 3.5 使用核SVM解决非线性问题
> 本章介绍了非线性可分问题，如何使用SVM核技巧解决非线性问题

支持向量机在机器学习爱好者中广受欢迎的另一原因是：它可以很容易地使用“核技巧”来解决非线性可分问题  
我们使用numpy中的logical_xor函数创建了一个经过“异或”操作的数据集，其中100个样本属于类别1，另外的100个样本被划定为类别-1  

In [ ]:
'''
python	numpy	Random sampling (numpy.random)	seed()
python	numpy	Random sampling (numpy.random)	randn()
python	numpy	Indexing	Other indexing options
python	numpy	Logic functions	logical_xor()
python	numpy	Indexing routines	where()
python	matplotlib	Pyplot function overview	scatter() marker
python	matplotlib	collections	label
'''
np.random.seed(0)
X_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(X_xor[:, 0] > 0, X_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)
plt.scatter(X_xor[y_xor==1, 0], X_xor[y_xor==1, 1],
           c='b', marker='x', label='1')
plt.scatter(X_xor[y_xor==-1, 0], X_xor[y_xor==-1, 1],
           c='r', marker='s', label='-1')
plt.ylim(-3.0)
plt.legend()
plt.show()

> 作者指出核方法的理念是通过函数转换后使得数据从非线性可分变得线性可分 5

核方法处理此类非线性可分数据的基本理念就是：通过映射函数$\varphi(·)$将样本的原始特征映射到一个使样本线性可分的更高维空间中，从而使得样本可分   
$$\phi(x_1,x_2)=(z_1,z_2,z_3)=(x_1,x_2,x_1^2+x_2^2)$$
如下图所示  
![3-7](../syn_pic/py_machine_learning/3-7.png)  
5  
### 使用核技巧在高维空间中发现分类超平面  
> 本篇解释了什么是核技巧，如何使用核技巧，以及如何在实践中调整gamma参数

为了使用SVM解决非线性问题，我们通过一个映射函数$\varphi(·)$将训练数据映射到更高维的特征空间，并在新的特征空间上训练一个线性SVM模型  
然后将同样的映射函数$\varphi(·)$应用于新的、未知数据上，进而使用新特征空间上的线性SVM模型对其进行分类  
映射方法面临的一个问题是：构建新的特征空间带来非常大的计算成本，特别是处理高维数据的时候。这时就用到了我们称作核技巧的方法  
为了降低两点之间内积精确计算阶段的成本耗费，我们定义了一个所谓的核函数:$k(x^{(i)},x^{(j)})=\varphi(x^{(i)})^T\varphi(x^{(j)})$  
5  
一个最广为使用的核函数就是径向基函数核(RBF kernel)或高斯核:  
$$k(x^{(i)},x^{(j)})=exp\left(-\frac{\|x^{(i)}-x^{(j)}\|^2}{2\sigma^2}\right)$$
简写为：  
$$k(x^{(i)},x^{(j)})=exp(-\gamma\|x^{(i)}-x^{(j)}\|^2)$$
这里,$\gamma=\frac{1}{2\sigma^2}$是待优化的自由参数 5  
粗略地说，"核"可解释为一对样本之间的“相似函数”。此处的负号将距离转换为相似性评分，而由于指数项的存在，使得相似性评分会介于0(差异巨大的样本)之间和1(完全相同的样本)  
现在我们尝试训练一个核SVM来对“异或”数据进行分类  

In [ ]:
'''
python	sklearn	Support Vector Machines	SVC() C
python	sklearn	Support Vector Machines	SVC() kernel
python	sklearn	Support Vector Machines	SVC() gammab
'''
svm = SVC(kernel='rbf', random_state=0, gamma=0.10, C=10.0)
svm.fit(X_xor, y_xor)
plot_decision_regions(X_xor, y_xor, classifier=svm)
plt.legend(loc='upper left')
plt.show()

参数$\gamma$可以理解为高斯球面的截止参数。**如果我们减小$\gamma$的值，将会增加受影响的训练样本的范围，这将导致决策边界更加宽松**  

In [ ]:
'''
5
'''
svm = SVC(kernel='rbf', random_state=0, gamma=0.2, C=1.0)
svm.fit(X_train_std, y_train)
plot_decision_regions(X_combined_std, y_combined, classifier=svm,
                     test_idx=range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc = 'upper left')
plt.show()

现在增加$\gamma$的值，并观察它对决策边界的影响  

In [ ]:
svm = SVC(kernel='rbf', random_state=0, gamma=100.0, C=1.0)
svm.fit(X_train_std, y_train)
plot_decision_regions(X_combined_std, y_combined, classifier=svm,
                     test_idx=range(105,150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal length [standardized]')
plt.legend(loc = 'upper left')
plt.show()

通过结果图像可以看到：使用一个相对较大的$\gamma$值，使得类别0和1的决策边界紧凑了许多  
虽然模型对训练数据的拟合非常好，但是类似的分类器对未知数据会有一个较高的泛化误差，这说明对$\gamma$的调优在控制过拟合方面也起到了重要作用 5  
# 3.6 决策树
> 介绍了决策树模型和使用方法，并进一步介绍了基于决策树的随机森林  

决策树吸引人的地方在于其模型的可解释性。我们可以把此模型看作将数据自顶向下进行划分，然后通过回答一系列问题来做出决策的方法  
![3-8](../syn_pic/py_machine_learning/3-8.png)  
*自顶向下的思维方式：数据分析中的维度细分、目标管理中的目标拆解*  
我们可以简单的为宽度设定一个临界值，并提出一个二元问题：“宽度是否达到2.8厘米？” 5  
使用决策树算法，我们从树根开始，基于可获得最大信息增益的特征来对数据进行划分，我们将在下一节详细介绍信息增益的概念  
通过迭代处理，在每个子节点上重复此划分过程，直到叶子节点  
在实际应用中，这可能会导致生成一颗深度很大且拥有众多节点的树，这样容易产生过拟合问题，由此，我们一般通过对树进行“剪枝”来限定树的最大深度 5  
## 3.6.1 最大化信息增益——获知尽可能准确的结果
> 介绍了信息增益的概念，进一步说明了如何实现信息增益最大化及其实现方式的区别  
  
为了在可获得最大信息增益的特征处进行节点划分，需要定义一个可通过树学习算法进行优化的目标函数。目标函数能够在每次划分时实现对信息增益的最大化，定义如下  
$$IG(D_p,f)=I(D_p)-\sum_{j=1}^m\frac{N_j}{N_p}I(D_j)$$
其中，f为将要进行划分的特征，$D_p$与$D_j$分别为父节点和第j个子节点，I为不纯度衡量标准，$N_p$为父节点中样本的数量，$N_j$为第j个子节点中样本的数量  
可见，信息增益只不过是父节点的不纯度与所有子节点不纯度总和之差——子节点的不纯度越低，信息增益越大 5  
然而，出于简化和缩小组合搜索空间的考虑，大多数库中实现的都是二叉决策树。这意味着每个父节点被划分为两个子节点：$D_{left}$和$D_{right}$  
$$IG(D_p,a)=I(D_p)-\frac{N_{left}}{N_p}I(D_{left})-\frac{N_{right}}{N_p}I(D_{right})$$
就目前来说，二叉决策树中常用的三个不纯度衡量标准或划分标准分别是：基尼系数(Gini index，$I_G$)、熵(entropy,$I_H$)，以及误分类率（classification error,$I_E$）。我们从非空类别$(p(i|t)\ne0)$熵的定义开始讲解：  
$$I_H(t)=-\sum_{i=1}^c{p(i|t)log_2p(i|t)}$$
5  
其中，$p(i|t)$为特定节点t中，属于类别c的样本占特定节点t中样本总数的比例。如果某一节点中所有的样本都属于同一类别，则其熵为0，当样本以相同的比例分属不同的类时，熵的值最大  
对二类别分类来说，如果类别均匀分布，即$p(i=1|t)=0.5且p(i=1|t)=0.5$时，熵为1  
因此，我们可以说在决策树中熵的准则就是使得互信息最大化  
### 问题：什么是互信息？
> 定义：衡量随机变量之间相互依赖程度的度量  
假如明天下雨是个随机事件，假如今晚有晚霞同样是个随机事件，那么这两个随机事件互相依赖的程度是  
当我们已知“今晚有晚霞“情况下，"明天下雨"带来的不确定性，与 不知道”今晚有晚霞“情况下，”明天下雨“带来的不确定性 之差 5  

### 问题：互信息和信息增益是什么关系？
> 说“互信息”的时候，两个随机变量的地位是相同的；说“信息增益”的时候，是把一个变量看成减小另一个变量不确定度的手段。但其实二者的数值是相等的 5  

直观地说，基尼系数可以理解为降低误分类可能性的标准：  
$$I_G(t)=\sum_{i=1}^c{p(i|t)(-p(i|t))}=1-\sum_{i=1}^c{p(i|t)^2}$$
5  
与熵类似，当所有类别是等比例分布时，基尼系数的值最大，例如在二类别分类中(c=2):  
$$1-\sum_{i=1}^c0.5^2=0.5$$
然而，**在实践中，基尼系数与熵通常会生成非常类似的结果，并不值得花费大量时间使用不纯度标准评估树的好坏**，而通常尝试使用不同的剪枝算法  
另一种不纯度衡量标准是误分类率：  
$$I_E=1-max\{p(i|t)\}$$
5  
**误分类率是一个对剪枝方法很有用的准则，但是不建议用于决策树的构建过程，因为它对节点中各类别样本数量的变动不敏感**  
![3-9](../syn_pic/py_machine_learning/3-9.png) 
在A和B两种划分情况下，使用误分类率得到的信息增益都是相同的（$IG_E=0.25$）:  
![3-10](../syn_pic/py_machine_learning/3-10.png) 
然后，在使用基尼系数时，与划分A（$IG_G=0.125$）相比，更倾向于使用B($IG_G=0.16$)的划分，因为这样子节点处的类别纯度相对更高 5  
同样，以熵为评估标准时，也是B($IG_G=0.31$)的信息增益大于A（IG_G=0.19）  
为了更直观地比较前面提到的三种不同的不纯度衡量标准，我们绘制样本属于类别1、概率介于[0,1]情况下不纯度的图像  

In [ ]:
'''
5
python	numpy	Exponents and logarithms	log2()
python	numpy	Mathematical functions	max()
python	matplotlib	axes	legend() loc
python	matplotlib	axes	legend() bbox_to_anchor
python	matplotlib	axes	legend() ncol
python	matplotlib	axes	legend() shadow
python	matplotlib	axes	axhline()
'''
def gini(p):
    return (p)*(1 - (p)) + (1 - p)*(1 - (1-p))
def entropy(p):
    return - p*np.log2(p) - (1 - p)*np.log2((1 - p))
def error(p):
    return 1 - np.max([p, 1 - p])
x = np.arange(0.0, 1.0, 0.01)
ent = [entropy(p) if p != 0 else None for p in x]
sc_ent = [e*0.5 if e else None for e in ent]
err = [error(i) for i in x]
fig = plt.figure()
ax = plt.subplot(111)
for i, lab, ls, c, in zip([ent, sc_ent, gini(x), err],
                         ['Entropy', 'Entropy(scaled)',
                         'Gini Impurity',
                         'Misclassification Error'],
                         ['-', '-', '--', '-.'],
                         ['black', 'lightgray',
                         'red', 'green', 'cyan']):
    line = ax.plot(x, i, label=lab,
                  linestyle=ls, lw=2, color=c)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15),
         ncol=3, shadow=False)
ax.axhline(y=0.5, linewidth=1, color='k', linestyle='--')
ax.axhline(y=1.0, linewidth=1, color='k', linestyle='--')
plt.ylim([0, 1.1])
plt.xlabel('p(i=1)')
plt.ylabel('Impurity Index')
plt.show()

## 3.6.2 构建决策树
> 介绍了如何借助sklearn实现决策树，以及决策树划分过程的可视化实现  

**必须注意：深度越大的决策树，决策边界也就越复杂，因而很容易产生过拟合现象**  
借助sklearn，我们将以熵作为不纯度度量标准，构建一颗最大深度为3的决策树。虽然特征缩放是处于可视化的目的，但在决策树算法中，这不是必须的  

In [ ]:
'''
python	sklearn	Decision Trees	DecisionTreeClassifier() criterion
python	sklearn	Decision Trees	DecisionTreeClassifier() max_depth
python	numpy	Array manipulation routines	vstack()
python	numpy	Array manipulation routines	hstack()
'''
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy',
                             max_depth=3, random_state=0)
tree.fit(X_train, y_train)
X_combined = np.vstack((X_train, X_test))
y_combined = np.hstack((y_train, y_test))
plot_decision_regions(X_combined, y_combined,
                     classifier=tree, test_idx=range(105,150))
plt.xlabel('petal length [cm]')
plt.ylabel('petal width [cm]')
plt.legend(loc='upper left')
plt.show()

sklearn一个吸引人的功能就是，它允许将训练后得到的决策树导出为.dot格式的文件，这使得我们可以使用GraphViz进行可视化处理 5  

In [ ]:
'''
python	sklearn	Decision Trees	export_graphviz() out_file
python	sklearn	Decision Trees	export_graphviz() feature_names
'''
from sklearn.tree import export_graphviz
export_graphviz(tree, 
               out_file='../tree.dot',
               feature_names=['petal length', 'petal width'])

在电脑上安装好GraphViz后，通过命令行窗口在保存tree.dot文件的路径处执行下面的命令，可将tree.dot文件转换为png文件  
> dot -Tpng tree.dot -o tree.png

In [ ]:
'''
python	sklearn	Decision Trees	plot_tree() feature_names
'''
from sklearn.tree import plot_tree
plt.figure(figsize=(10,8))
plot_tree(tree,feature_names=['petal length', 'petal width'])
plt.show()

通过观察GraphViz创建的图像，可以很好地回溯决策树在训练数据集上对各节点进行划分的过程 5  
## 3.6.3 通过随机森林将弱分类器集成为强分类器
> 本篇介绍了随机森林以及如何使用训练及调优随机森林模型  

直观上，随机森林可以视为多颗决策树的集成。**集成学习的基本理念就是将弱分类器集成为鲁棒性更强的模型，集成后具备更好的泛化误差，不易产生过拟合现象** 
随机森林算法可以概况为四个简单的步骤：  
1. 使用bootstrap抽样方法随机选择N个样本用于训练
2. 使用第1步选定的样本构造一颗决策树，节点划分规则如下： 5
    1. 不重复地随机选择d个特征
    2. 根据目标函数的要求，如最大化信息增益，使用选定的特征对节点进行划分  
3. 重复上述过程1~2000次
4. 汇总每颗决策树的类标进行多数投票（第7章中会详细介绍）  

虽然随机森林没有决策树那样良好的可解释性，但其显著的优势在于不必担心超参值的选择。**我们通常不需要对随机森林进行剪枝，因为相对于单颗决策树来说，集成模型对噪声的鲁棒性更好** 5  
在实践中，我们**真正需要关心的参数是为构建随机森林所需的决策树数量**。通常情况下，决策树的数量越多，随机森林整体的分类表现就越好，但同时也相应地增加了计算成本  
尽管在实践中不常见，但是随机森林中可优化的其他超参分别是：bootstrap抽样的数量以及在节点划分中使用的特征数量，它们所采用的技术将在第5章中讨论  
通过选择bootstrap抽样中样本数量n，我们可以控制随机森林的偏差与方差权衡的问题。如果n值较大，就降低了随机性，会导致过拟合，反之n值较小则会导致欠拟合  
包括RandomForestClassifier在内的大多数对随机森林的实现中，bootstrap抽样的数量一般与原始训练集中样本的数量相同，因为这样在这种偏差与方差方面一般会有一个好的均衡结果  
而对于在每次节点划分中用到的特征数量m，我们选择一个比训练集中特征总量小的值。常用的默认值一般是$d=\sqrt{m}$，其中m是训练集中特征的总量 5  

In [ ]:
'''
python	sklearn	Ensemble Methods	RandomForestClassifier() n_estimators
python	sklearn	Ensemble Methods	RandomForestClassifier() criterion
python	sklearn	Ensemble Methods	RandomForestClassifier() n_jobs
'''
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy',
                               n_estimators=10,
                               random_state=1,
                               n_jobs=-1)
forest.fit(X_train, y_train)
plot_decision_regions(X_combined, y_combined,
                     classifier=forest, test_idx=range(105,150))
plt.xlabel('petal length [cm]')
plt.ylabel('petal width [cm]')
plt.legend(loc='upper left')
plt.show()